In [41]:
import matplotlib
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import matplotlib.colorbar as mcolorbar
import matplotlib.pyplot as plt
from matplotlib.patches import Wedge
%matplotlib widget

import seaborn as sns
import sklearn
import pandas as pd
import numpy as np

def plot_conf(type):
    choices = {'back': 'dodgerblue', 'backd': 'navy', 'frontal': 'seagreen', 'frontald': 'lightseagreen'}
    titles = {'sider': 'Side Right',
               'backdr': 'Back Diagonal Right',
               'back': 'Back',
               'backdl': 'Back Diagonal Left',
               'sidel': 'Side Left',
               'frontaldl': 'Frontal Diagonal Left',
               'frontal': 'Frontal',
               'frontaldr': 'Frontal Diagonal Right',
               'pairinter':'Pair view intersection',
               'formations': 'Group movement stops'
               }
    ranges = {'sider': '[-22.5°,0°),[0°,22.5°)',
               'backdr': '[22.5°,67.5°)',
               'back': '[67.5°,112.5°)',
               'backdl': '[112.5°,157.5°)',
               'sidel': '[157.5°,180°),(-180°,157.5°]',
               'frontaldl': '[-157.5°, -112.5°)',
               'frontal': '[-112.5°, -67.5°)',
               'frontaldr': '[-67.5°, -22.5°)'
               }
        
    shade_color = choices.get(type, 'purple')
    title = titles.get(type, 'Body')
    range_title=ranges.get(type,' ')
    
    return shade_color, title, range_title
    

    
def angle_cloud(data_kinect, shade_color, title,range_title, type):
  
    U = np.cos((data_kinect['re_body_angle']) )
    V = np.sin((data_kinect['re_body_angle']) )

    sns.set()

    norm = matplotlib.colors.Normalize()
    norm.autoscale(data_kinect['re_body_angle'])
    cm = matplotlib.cm.viridis

    sm = matplotlib.cm.ScalarMappable(cmap=cm, norm=norm)
    sm.set_array([])

    fig, ax = plt.subplots()

    if type==' outliers':

        segments = list(zip(zip(data_kinect["shr_x"], data_kinect["shr_y"]), zip(data_kinect["shl_x"], data_kinect["shl_y"])))
        lines = []
        for item in segments:
            lines.append([item[0], item[1]])

        lc = mc.LineCollection(lines, linewidths=1.0, linestyles='dashed', alpha=0.7)

        ax.add_collection(lc)
        sns.scatterplot(x="shr_x", y="shr_y", data=data_kinect, alpha=0.7,color='red', s=50)
        sns.scatterplot(x="shl_x", y="shl_y", data=data_kinect, alpha=0.7,color='orange', s=50)

    ax.quiver(data_kinect['shl_x'], data_kinect['shl_y'], U, V, angles=data_kinect['re_body_angle'], color=cm(norm(data_kinect['re_body_angle'])), units='xy',pivot='middle')

    ax.axis(xmin=-3, xmax=3)
    ax.axis(ymin=-0.5, ymax=4.5)

    fov = Wedge(center=(0, 0), r=4.895, theta1=55, theta2=125, color=shade_color, alpha=0.05)
    ax.add_artist(fov)

    plt.ylabel('(Y) Distance from Origin')
    plt.xlabel('(X) Distance from Origin')
    plt.title('Body Orientation' +type+': '+title+'\n Accepted angle range '+range_title)

    cax, _ = mcolorbar.make_axes(plt.gca())
    cb = mcolorbar.ColorbarBase(cax, cmap=matplotlib.cm.viridis, norm=norm)
    cb.set_label('Body Orientation angle')

    return plt

data_kinect = pd.read_csv("frontaldl_processed_data_frontaldl.csv",sep=';') 
data_kinect.reset_index(drop=True)
print(data_kinect)

type='frontaldl'
shade_color, title, range_title= plot_conf(type)
angle_cloud(data_kinect, shade_color, title, range_title, type).show()

      Unnamed: 0     ID_exp                    date_exp description  Kheight  \
0              0  AW_221120  2020-11-22 14:56:17.129877   frontaldl    1.833   
1              3  AW_221120  2020-11-22 14:56:17.791148   frontaldl    1.833   
2              5  AW_221120  2020-11-22 14:56:18.224701   frontaldl    1.833   
3              8  AW_221120  2020-11-22 14:56:18.889080   frontaldl    1.833   
4             13  AW_221120  2020-11-22 14:56:19.986436   frontaldl    1.833   
...          ...        ...                         ...         ...      ...   
1617        1980  AW_171220  2020-12-17 05:12:45.384457   frontaldl    1.833   
1618        1981  AW_171220  2020-12-17 05:12:45.603111   frontaldl    1.833   
1619        1982  AW_171220  2020-12-17 05:12:45.835644   frontaldl    1.833   
1620        1983  AW_171220  2020-12-17 05:12:46.069020   frontaldl    1.833   
1621        1984  AW_171220  2020-12-17 05:12:46.278443   frontaldl    1.833   

       Kx   Ky   Kz  tilx  tily  ...  r

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …